In [1]:
from dash import Dash, html, dcc, Input, Output, State
import pandas as pd
import pickle


with open("Export\Dizionario Metriche\Minimized Version\dicMetric_Privacy_Completed.pkl", "rb") as file:
    dicMetric_Privacy_Completed = pickle.load(file)

# Creare il DataFrame di partenza
df = pd.DataFrame(dicMetric_Privacy_Completed)

# Creare l'app Dash
app = Dash(__name__)

# Layout dell'app
app.layout = html.Div([
    html.H1("Privacy Metrics Form", style={"textAlign": "center"}),

    # Tabella dinamica con input per impatti
    html.Div([
        html.Table([
            html.Tr([html.Th("Metric"), html.Th("Value"), html.Th("Impact (0-10)")]),
            *[html.Tr([
                html.Td(df.iloc[i]['Metric']),
                html.Td(f"{df.iloc[i]['Valore']:.2f}", style={"textAlign": "center"} ),
                html.Td(dcc.Input(
                    id=f"impact-{i}", type="number", min=0, max=10, step=1, value=0
                ), style={"textAlign": "center"})
            ]) for i in range(len(df))]
        ], style={"width": "80%", "margin": "auto", "border": "1px solid black"})
    ], style={"marginBottom": "20px"}),

    # Bottone per calcolare la metrica globale
    html.Div([
        html.Button("Global Metric Calculation", id="calcola-button", n_clicks=0, style={"marginTop": "10px"}),
        html.Div(id="output-globale", style={"marginTop": "20px", "fontSize": "18px"})
    ], style={"textAlign": "center"})
])

# Callback per calcolare la metrica globale
@app.callback(
    Output("output-globale", "children"),
    Input("calcola-button", "n_clicks"),
    [State(f"impact-{i}", "value") for i in range(len(df))]
)
def calcola_metrica_globale(n_clicks, *impacts):
    if n_clicks == 0:
        return "Enter the impact values ​​and press the button to calculate."

    # Organizzare gli impatti
    impacts = list(impacts)
    totale_pesato = 0
    somma_impatti = 0

    for i in range(len(df)):
        valore_metrica = df.iloc[i]['Valore']
        impatto = impacts[i]
        totale_pesato += valore_metrica * impatto
        somma_impatti += impatto

    # Calcolo della metrica globale
    if somma_impatti > 0:
        metrica_globale = totale_pesato / somma_impatti
        return f"Global Metric: {metrica_globale:.2f}"
    else:
        return "Error: The sum of the impacts is zero. Enter at least one impact greater than zero."

# Avvio dell'app (solo se eseguito in un ambiente compatibile)
if __name__ == "__main__":
    app.run_server(debug=True)
